# Notebook 6: 🔧 Workshop - Building a Filesystem MCP Server

## Best Practices for Agentic AI using MCP

---

**Series:** MCP Server Best Practices - Agentic AI Workflows  
**Notebook:** 6 of 13  
**Level:** Intermediate (Hands-On Workshop)  
**Duration:** ~90 minutes

---

## 🎯 Workshop Objective

In this hands-on workshop, you will **build a complete Filesystem MCP Server** from scratch that allows Claude to:

- 📂 List directories and files
- 📖 Read file contents
- ✍️ Create and write files
- 🔍 Search files by name or content
- 📊 Get file metadata

All with **proper security controls** to limit access to specified directories.

## Workshop Structure

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    WORKSHOP ROADMAP                                      │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   Part 1: Project Setup (15 min)                                        │
│   └─► Create project structure, install dependencies                    │
│                                                                          │
│   Part 2: Security Layer (20 min)                                       │
│   └─► Build path validation and access controls                         │
│                                                                          │
│   Part 3: Core Tools (30 min)                                           │
│   └─► Implement list, read, write, search tools                         │
│                                                                          │
│   Part 4: Resources (15 min)                                            │
│   └─► Add file resources with URI templates                             │
│                                                                          │
│   Part 5: Testing & Integration (10 min)                                │
│   └─► Test with MCP Inspector and Claude Desktop                        │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## Part 1: Project Setup

### 1.1 Create Project Structure

In your PowerShell terminal, run:

```powershell
# Create project directory
mkdir filesystem-mcp-server
cd filesystem-mcp-server

# Create virtual environment
python -m venv venv
.\venv\Scripts\Activate.ps1

# Install dependencies
pip install fastmcp pydantic

# Create project files
New-Item -ItemType File -Name "server.py"
New-Item -ItemType File -Name "security.py"
New-Item -ItemType File -Name "config.py"
```

### 1.2 Project Structure

```
filesystem-mcp-server/
├── venv/
├── server.py          # Main MCP server
├── security.py        # Path validation & access control
├── config.py          # Configuration settings
└── test_data/         # Test directory (we'll create this)
    ├── documents/
    ├── notes/
    └── sample.txt
```

In [ ]:
# Let's define what we're building
import json

server_spec = {
    "name": "FilesystemMCP",
    "version": "1.0.0",
    "description": "Secure filesystem access for AI agents",
    "tools": [
        {"name": "list_directory", "description": "List files and folders in a directory"},
        {"name": "read_file", "description": "Read contents of a file"},
        {"name": "write_file", "description": "Create or overwrite a file"},
        {"name": "search_files", "description": "Search for files by name pattern"},
        {"name": "get_file_info", "description": "Get metadata about a file"}
    ],
    "resources": [
        {"uri": "file:///{path}", "description": "Access any allowed file by path"}
    ],
    "security": {
        "path_validation": "Restrict to allowed directories",
        "symlink_protection": "Prevent symlink attacks",
        "size_limits": "Limit file sizes for read/write"
    }
}

print("Filesystem MCP Server Specification:")
print("=" * 60)
print(json.dumps(server_spec, indent=2))

## Part 2: Security Layer

### 🔐 Security is CRITICAL for filesystem access!

We must prevent:
- **Path traversal attacks** (`../../../etc/passwd`)
- **Symlink escapes** (symlinks pointing outside allowed dirs)
- **Unrestricted access** (accessing system files)

### 2.1 Create `security.py`

In [ ]:
# security.py - Copy this to your project
SECURITY_CODE = '''
"""Security module for Filesystem MCP Server.

Provides path validation and access control to ensure
the server only accesses allowed directories.
"""

import os
from pathlib import Path
from typing import List, Optional
import logging

logger = logging.getLogger(__name__)


class PathSecurityError(Exception):
    """Raised when a path access is denied."""
    pass


class SecurePathValidator:
    """
    Validates file paths to ensure they are within allowed directories.
    
    Security features:
    - Resolves all paths to absolute paths
    - Checks against allowlist of directories
    - Prevents symlink escapes
    - Blocks path traversal attacks
    """
    
    def __init__(
        self,
        allowed_directories: List[str],
        follow_symlinks: bool = False,
        max_file_size_mb: float = 10.0
    ):
        """
        Initialize the path validator.
        
        Args:
            allowed_directories: List of directory paths that are allowed
            follow_symlinks: Whether to follow symlinks (False is safer)
            max_file_size_mb: Maximum file size for read/write operations
        """
        self.allowed_directories = []
        self.follow_symlinks = follow_symlinks
        self.max_file_size_bytes = int(max_file_size_mb * 1024 * 1024)
        
        # Resolve and validate allowed directories
        for dir_path in allowed_directories:
            resolved = Path(dir_path).resolve()
            if resolved.exists() and resolved.is_dir():
                self.allowed_directories.append(resolved)
                logger.info(f"Allowed directory: {resolved}")
            else:
                logger.warning(f"Directory does not exist, skipping: {dir_path}")
        
        if not self.allowed_directories:
            raise ValueError("No valid allowed directories provided!")
    
    def validate_path(self, path: str) -> Path:
        """
        Validate and resolve a path.
        
        Args:
            path: The path to validate
            
        Returns:
            Resolved Path object if valid
            
        Raises:
            PathSecurityError: If path is not allowed
        """
        try:
            # Convert to Path object
            target_path = Path(path)
            
            # Resolve to absolute path (handles ../ etc)
            if self.follow_symlinks:
                resolved = target_path.resolve()
            else:
                # Use strict=False to handle non-existent paths
                resolved = target_path.resolve()
                # Check if it\'s a symlink pointing outside
                if target_path.is_symlink():
                    link_target = target_path.resolve()
                    if not self._is_within_allowed(link_target):
                        raise PathSecurityError(
                            f"Symlink points outside allowed directories: {path}"
                        )
            
            # Check if path is within allowed directories
            if not self._is_within_allowed(resolved):
                raise PathSecurityError(
                    f"Path is outside allowed directories: {path}"
                )
            
            return resolved
            
        except PathSecurityError:
            raise
        except Exception as e:
            raise PathSecurityError(f"Invalid path: {path} - {e}")
    
    def _is_within_allowed(self, path: Path) -> bool:
        """
        Check if a resolved path is within any allowed directory.
        """
        for allowed_dir in self.allowed_directories:
            try:
                path.relative_to(allowed_dir)
                return True
            except ValueError:
                continue
        return False
    
    def validate_file_for_read(self, path: str) -> Path:
        """
        Validate a file path for reading.
        
        Additional checks:
        - File must exist
        - Must be a file (not directory)
        - Must be within size limit
        """
        resolved = self.validate_path(path)
        
        if not resolved.exists():
            raise PathSecurityError(f"File does not exist: {path}")
        
        if not resolved.is_file():
            raise PathSecurityError(f"Path is not a file: {path}")
        
        file_size = resolved.stat().st_size
        if file_size > self.max_file_size_bytes:
            raise PathSecurityError(
                f"File too large ({file_size} bytes > {self.max_file_size_bytes} bytes): {path}"
            )
        
        return resolved
    
    def validate_file_for_write(self, path: str, content_size: int = 0) -> Path:
        """
        Validate a file path for writing.
        
        Additional checks:
        - Parent directory must exist
        - Content size must be within limit
        """
        resolved = self.validate_path(path)
        
        # Check parent directory exists
        if not resolved.parent.exists():
            raise PathSecurityError(
                f"Parent directory does not exist: {resolved.parent}"
            )
        
        # Check content size
        if content_size > self.max_file_size_bytes:
            raise PathSecurityError(
                f"Content too large ({content_size} bytes > {self.max_file_size_bytes} bytes)"
            )
        
        return resolved
    
    def validate_directory(self, path: str) -> Path:
        """
        Validate a directory path.
        """
        resolved = self.validate_path(path)
        
        if not resolved.exists():
            raise PathSecurityError(f"Directory does not exist: {path}")
        
        if not resolved.is_dir():
            raise PathSecurityError(f"Path is not a directory: {path}")
        
        return resolved
'''

print("security.py:")
print("=" * 60)
print(SECURITY_CODE)

### 2.2 Test the Security Module

In [ ]:
# Test security module (conceptual - run in your actual project)
SECURITY_TEST = '''
# Run this in your project to test security

from security import SecurePathValidator, PathSecurityError

# Create validator with allowed directory
validator = SecurePathValidator(
    allowed_directories=["C:\\\\Users\\\\YourName\\\\Documents\\\\allowed_folder"],
    max_file_size_mb=5.0
)

# Test cases
test_paths = [
    # Should PASS - within allowed directory
    "C:\\\\Users\\\\YourName\\\\Documents\\\\allowed_folder\\\\file.txt",
    
    # Should FAIL - path traversal attempt
    "C:\\\\Users\\\\YourName\\\\Documents\\\\allowed_folder\\\\..\\\\..\\\\secret.txt",
    
    # Should FAIL - outside allowed directory
    "C:\\\\Windows\\\\System32\\\\config",
]

for path in test_paths:
    try:
        validated = validator.validate_path(path)
        print(f"✅ ALLOWED: {path}")
    except PathSecurityError as e:
        print(f"❌ BLOCKED: {path}")
        print(f"   Reason: {e}")
'''

print("Security Test Script:")
print("=" * 60)
print(SECURITY_TEST)

## Part 3: Core Tools

### 3.1 Create `config.py`

In [ ]:
# config.py
CONFIG_CODE = '''
"""Configuration for Filesystem MCP Server."""

import os
from pathlib import Path

# Allowed directories - CUSTOMIZE THIS!
# Add the directories you want Claude to access
ALLOWED_DIRECTORIES = [
    # Example: Your Documents folder
    os.path.expanduser("~/Documents/mcp-workspace"),
    
    # Example: A specific project folder
    # "C:\\Users\\YourName\\Projects",
    
    # Example: Notes folder
    # os.path.expanduser("~/Notes"),
]

# Security settings
MAX_FILE_SIZE_MB = 10.0  # Maximum file size for read/write
FOLLOW_SYMLINKS = False  # Safer to keep False

# Server settings
SERVER_NAME = "FilesystemMCP"
SERVER_VERSION = "1.0.0"

# File type restrictions (optional)
ALLOWED_EXTENSIONS = None  # None = all allowed, or set like [".txt", ".md", ".json"]

# Patterns to ignore when listing
IGNORE_PATTERNS = [
    "__pycache__",
    ".git",
    "node_modules",
    ".venv",
    "venv",
    ".DS_Store",
    "Thumbs.db"
]
'''

print("config.py:")
print("=" * 60)
print(CONFIG_CODE)

### 3.2 Create `server.py` - Main Server

In [ ]:
# server.py - The complete MCP server
SERVER_CODE = '''
#!/usr/bin/env python3
"""Filesystem MCP Server

A secure MCP server that provides filesystem access to AI agents.

Usage:
    python server.py                    # Run with STDIO transport
    python server.py --transport sse    # Run with HTTP+SSE transport
"""

import os
import sys
import fnmatch
import logging
from pathlib import Path
from datetime import datetime
from typing import List, Optional

from fastmcp import FastMCP
from pydantic import BaseModel, Field

from security import SecurePathValidator, PathSecurityError
from config import (
    ALLOWED_DIRECTORIES,
    MAX_FILE_SIZE_MB,
    FOLLOW_SYMLINKS,
    SERVER_NAME,
    SERVER_VERSION,
    IGNORE_PATTERNS
)

# Configure logging to stderr (CRITICAL for STDIO!)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    stream=sys.stderr
)
logger = logging.getLogger(__name__)

# Initialize security validator
try:
    path_validator = SecurePathValidator(
        allowed_directories=ALLOWED_DIRECTORIES,
        follow_symlinks=FOLLOW_SYMLINKS,
        max_file_size_mb=MAX_FILE_SIZE_MB
    )
    logger.info(f"Initialized with {len(path_validator.allowed_directories)} allowed directories")
except ValueError as e:
    logger.error(f"Failed to initialize: {e}")
    sys.exit(1)

# Create MCP server
mcp = FastMCP(SERVER_NAME)


# ═══════════════════════════════════════════════════════════════════════════
# TOOLS
# ═══════════════════════════════════════════════════════════════════════════

@mcp.tool()
def list_directory(
    path: str,
    include_hidden: bool = False,
    recursive: bool = False,
    max_depth: int = 3
) -> dict:
    """
    List files and directories at the specified path.
    
    Args:
        path: Directory path to list
        include_hidden: Include hidden files (starting with .)
        recursive: List subdirectories recursively
        max_depth: Maximum recursion depth (default 3)
    
    Returns:
        Dictionary containing files, directories, and metadata
    """
    try:
        validated_path = path_validator.validate_directory(path)
        logger.info(f"Listing directory: {validated_path}")
        
        def should_ignore(name: str) -> bool:
            if not include_hidden and name.startswith("."):
                return True
            return any(fnmatch.fnmatch(name, pattern) for pattern in IGNORE_PATTERNS)
        
        def list_dir(dir_path: Path, current_depth: int = 0) -> dict:
            files = []
            directories = []
            
            try:
                for item in sorted(dir_path.iterdir()):
                    if should_ignore(item.name):
                        continue
                    
                    if item.is_file():
                        files.append({
                            "name": item.name,
                            "size": item.stat().st_size,
                            "modified": datetime.fromtimestamp(
                                item.stat().st_mtime
                            ).isoformat()
                        })
                    elif item.is_dir():
                        dir_info = {"name": item.name}
                        if recursive and current_depth < max_depth:
                            dir_info["contents"] = list_dir(item, current_depth + 1)
                        directories.append(dir_info)
            except PermissionError:
                logger.warning(f"Permission denied: {dir_path}")
            
            return {"files": files, "directories": directories}
        
        result = list_dir(validated_path)
        result["path"] = str(validated_path)
        result["total_files"] = len(result["files"])
        result["total_directories"] = len(result["directories"])
        
        return result
        
    except PathSecurityError as e:
        logger.warning(f"Security error in list_directory: {e}")
        return {"error": str(e)}
    except Exception as e:
        logger.error(f"Error in list_directory: {e}")
        return {"error": f"Failed to list directory: {e}"}


@mcp.tool()
def read_file(path: str, encoding: str = "utf-8") -> dict:
    """
    Read the contents of a file.
    
    Args:
        path: Path to the file to read
        encoding: File encoding (default: utf-8)
    
    Returns:
        Dictionary with file content and metadata
    """
    try:
        validated_path = path_validator.validate_file_for_read(path)
        logger.info(f"Reading file: {validated_path}")
        
        stat = validated_path.stat()
        
        try:
            content = validated_path.read_text(encoding=encoding)
        except UnicodeDecodeError:
            # Try reading as binary if text fails
            content = f"[Binary file - {stat.st_size} bytes]"
        
        return {
            "path": str(validated_path),
            "content": content,
            "size": stat.st_size,
            "modified": datetime.fromtimestamp(stat.st_mtime).isoformat(),
            "encoding": encoding
        }
        
    except PathSecurityError as e:
        logger.warning(f"Security error in read_file: {e}")
        return {"error": str(e)}
    except Exception as e:
        logger.error(f"Error in read_file: {e}")
        return {"error": f"Failed to read file: {e}"}


@mcp.tool()
def write_file(
    path: str,
    content: str,
    encoding: str = "utf-8",
    create_directories: bool = False
) -> dict:
    """
    Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
        encoding: File encoding (default: utf-8)
        create_directories: Create parent directories if they don\'t exist
    
    Returns:
        Dictionary with write result and file info
    """
    try:
        content_bytes = content.encode(encoding)
        validated_path = path_validator.validate_file_for_write(
            path, len(content_bytes)
        )
        
        logger.info(f"Writing file: {validated_path}")
        
        # Create parent directories if requested
        if create_directories and not validated_path.parent.exists():
            # Validate parent path too
            path_validator.validate_path(str(validated_path.parent))
            validated_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Write the file
        validated_path.write_text(content, encoding=encoding)
        
        stat = validated_path.stat()
        
        return {
            "success": True,
            "path": str(validated_path),
            "size": stat.st_size,
            "modified": datetime.fromtimestamp(stat.st_mtime).isoformat()
        }
        
    except PathSecurityError as e:
        logger.warning(f"Security error in write_file: {e}")
        return {"success": False, "error": str(e)}
    except Exception as e:
        logger.error(f"Error in write_file: {e}")
        return {"success": False, "error": f"Failed to write file: {e}"}


@mcp.tool()
def search_files(
    directory: str,
    pattern: str,
    search_content: bool = False,
    max_results: int = 50
) -> dict:
    """
    Search for files by name pattern or content.
    
    Args:
        directory: Directory to search in
        pattern: Glob pattern for filename (e.g., "*.txt", "report*")
        search_content: If True, search file contents for pattern
        max_results: Maximum number of results to return
    
    Returns:
        Dictionary with matching files
    """
    try:
        validated_path = path_validator.validate_directory(directory)
        logger.info(f"Searching in {validated_path} for pattern: {pattern}")
        
        matches = []
        
        for item in validated_path.rglob("*"):
            if len(matches) >= max_results:
                break
            
            if not item.is_file():
                continue
            
            # Check if item is still within allowed directories
            try:
                path_validator.validate_path(str(item))
            except PathSecurityError:
                continue
            
            # Match by filename
            if fnmatch.fnmatch(item.name, pattern):
                matches.append({
                    "path": str(item),
                    "name": item.name,
                    "size": item.stat().st_size,
                    "match_type": "filename"
                })
            # Optionally search content
            elif search_content:
                try:
                    content = item.read_text(errors="ignore")
                    if pattern.lower() in content.lower():
                        matches.append({
                            "path": str(item),
                            "name": item.name,
                            "size": item.stat().st_size,
                            "match_type": "content"
                        })
                except Exception:
                    pass  # Skip files that can\'t be read
        
        return {
            "directory": str(validated_path),
            "pattern": pattern,
            "matches": matches,
            "total_matches": len(matches),
            "truncated": len(matches) >= max_results
        }
        
    except PathSecurityError as e:
        logger.warning(f"Security error in search_files: {e}")
        return {"error": str(e)}
    except Exception as e:
        logger.error(f"Error in search_files: {e}")
        return {"error": f"Failed to search: {e}"}


@mcp.tool()
def get_file_info(path: str) -> dict:
    """
    Get detailed metadata about a file.
    
    Args:
        path: Path to the file
    
    Returns:
        Dictionary with file metadata
    """
    try:
        validated_path = path_validator.validate_path(path)
        logger.info(f"Getting info for: {validated_path}")
        
        if not validated_path.exists():
            return {"error": f"Path does not exist: {path}"}
        
        stat = validated_path.stat()
        
        info = {
            "path": str(validated_path),
            "name": validated_path.name,
            "type": "directory" if validated_path.is_dir() else "file",
            "size": stat.st_size,
            "created": datetime.fromtimestamp(stat.st_ctime).isoformat(),
            "modified": datetime.fromtimestamp(stat.st_mtime).isoformat(),
            "accessed": datetime.fromtimestamp(stat.st_atime).isoformat(),
        }
        
        if validated_path.is_file():
            info["extension"] = validated_path.suffix
            # Try to detect if it\'s a text file
            try:
                validated_path.read_text()
                info["is_text"] = True
            except UnicodeDecodeError:
                info["is_text"] = False
        
        return info
        
    except PathSecurityError as e:
        logger.warning(f"Security error in get_file_info: {e}")
        return {"error": str(e)}
    except Exception as e:
        logger.error(f"Error in get_file_info: {e}")
        return {"error": f"Failed to get file info: {e}"}


# ═══════════════════════════════════════════════════════════════════════════
# RESOURCES
# ═══════════════════════════════════════════════════════════════════════════

@mcp.resource("file:///{path}")
def read_file_resource(path: str) -> str:
    """
    Resource handler for file:/// URIs.
    
    Allows reading files as MCP resources.
    """
    try:
        validated_path = path_validator.validate_file_for_read(path)
        return validated_path.read_text()
    except Exception as e:
        return f"Error reading file: {e}"


# ═══════════════════════════════════════════════════════════════════════════
# MAIN
# ═══════════════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    import argparse
    
    parser = argparse.ArgumentParser(description=f"{SERVER_NAME} v{SERVER_VERSION}")
    parser.add_argument(
        "--transport",
        choices=["stdio", "sse"],
        default="stdio",
        help="Transport mechanism (default: stdio)"
    )
    parser.add_argument(
        "--port",
        type=int,
        default=8000,
        help="Port for SSE transport (default: 8000)"
    )
    
    args = parser.parse_args()
    
    logger.info(f"Starting {SERVER_NAME} v{SERVER_VERSION}")
    logger.info(f"Transport: {args.transport}")
    
    if args.transport == "sse":
        mcp.run(transport="sse", host="0.0.0.0", port=args.port)
    else:
        mcp.run(transport="stdio")
'''

print("server.py (Complete MCP Server):")
print("=" * 60)
print(SERVER_CODE)

## Part 4: Testing Your Server

### 4.1 Create Test Data

```powershell
# In your project directory
mkdir test_data
mkdir test_data\documents
mkdir test_data\notes

# Create sample files
echo "Hello, this is a test file!" > test_data\sample.txt
echo "# Meeting Notes`nDiscussed MCP servers" > test_data\notes\meeting.md
echo '{"name": "test", "version": "1.0"}' > test_data\documents\config.json
```

### 4.2 Update config.py

In [ ]:
# Update ALLOWED_DIRECTORIES in config.py to include test_data
CONFIG_UPDATE = '''
# In config.py, update ALLOWED_DIRECTORIES:

import os

# Get the directory where this config file is located
BASE_DIR = os.path.dirname(os.path.abspath(__file__))

ALLOWED_DIRECTORIES = [
    os.path.join(BASE_DIR, "test_data"),  # Test directory
    # Add more directories as needed
]
'''

print("Update config.py with:")
print(CONFIG_UPDATE)

### 4.3 Test with MCP Inspector

```powershell
# Install MCP Inspector if not already installed
npm install -g @modelcontextprotocol/inspector

# Run inspector with your server
mcp-inspector python server.py

# Opens browser at http://localhost:6274
```

### 4.4 Configure Claude Desktop

Add to `%APPDATA%\Claude\claude_desktop_config.json`:

In [ ]:
# Claude Desktop configuration
import json

claude_config = {
    "mcpServers": {
        "filesystem": {
            "command": "python",
            "args": [
                "C:\\Users\\YourName\\filesystem-mcp-server\\server.py"
            ],
            "env": {
                "PYTHONUNBUFFERED": "1"
            }
        }
    }
}

print("Claude Desktop Configuration:")
print("=" * 60)
print(json.dumps(claude_config, indent=2))
print("\n📁 Save to: %APPDATA%\\Claude\\claude_desktop_config.json")
print("⚠️ Restart Claude Desktop after saving!")

## Part 5: Using Your Server with Claude

Once configured, you can ask Claude things like:

### Example Interactions

```
User: "List all files in my test_data directory"
Claude: [calls list_directory tool]

User: "Read the contents of sample.txt"
Claude: [calls read_file tool]

User: "Create a new file called ideas.md with a list of project ideas"
Claude: [calls write_file tool]

User: "Find all .json files in my workspace"
Claude: [calls search_files tool]

User: "What's the size and modification date of config.json?"
Claude: [calls get_file_info tool]
```

## 🎉 Workshop Complete!

You've built a **complete, secure Filesystem MCP Server** that:

### ✅ Features Implemented
- **5 Tools**: list_directory, read_file, write_file, search_files, get_file_info
- **1 Resource Template**: `file:///{path}` for direct file access
- **Security Layer**: Path validation, symlink protection, size limits
- **Configurable**: Easy to customize allowed directories

### 🔐 Security Measures
- Path traversal protection
- Directory allowlist enforcement
- Symlink escape prevention
- File size limits
- Comprehensive logging

### 📚 Next Steps
1. Add more tools (delete, move, copy files)
2. Add file watching (resource subscriptions)
3. Add prompts for common workflows
4. Deploy with HTTP+SSE for remote access

## Bonus: Additional Tools to Implement

Challenge yourself by adding these tools:

In [ ]:
# Bonus tools to implement
BONUS_TOOLS = '''
@mcp.tool()
def create_directory(path: str) -> dict:
    """Create a new directory."""
    # Your implementation
    pass

@mcp.tool()
def delete_file(path: str, confirm: bool = False) -> dict:
    """Delete a file (requires confirmation)."""
    # Your implementation
    pass

@mcp.tool()
def move_file(source: str, destination: str) -> dict:
    """Move a file from source to destination."""
    # Your implementation
    pass

@mcp.tool()
def copy_file(source: str, destination: str) -> dict:
    """Copy a file from source to destination."""
    # Your implementation
    pass

@mcp.tool()
def append_to_file(path: str, content: str) -> dict:
    """Append content to an existing file."""
    # Your implementation
    pass
'''

print("Bonus Challenge - Implement these tools:")
print("=" * 60)
print(BONUS_TOOLS)

---

## References

- [Anthropic Filesystem MCP Server](https://github.com/anthropics/mcp-servers/tree/main/src/filesystem)
- [FastMCP Documentation](https://github.com/jlowin/fastmcp)
- [Python pathlib Documentation](https://docs.python.org/3/library/pathlib.html)

---

**Next Notebook:** [07_MCP_Server_Best_Practices.ipynb](./07_MCP_Server_Best_Practices.ipynb)